# ZDDのグラフアルゴリズム

## サンプル
　__今回考えるのは、以下の図。幅優先探索で、各頂点に名前をつける__

<img src="../image/graphillion/sample.png" width=40%>

In [1]:
# 必要なモジュールのインポート
import collections
import copy

In [2]:
edges = [("e0","e1"),("e0","e2"),("e1","e3"),("e1","e4"),("e2","e5"),("e3","e5"),("e4","e5")]
# 頂点の集合
vertex_set = ["e0","e1","e2","e3","e4","e5"]
# mate配列

In [3]:
# 入次数を辞書で記録
counter = collections.Counter([edge[0] for edge in edges])

```
# 始点("e1")で条件をしぼって終点を抽出する
[edges[i][1] for i in range(len(edges)) if edges[i][0] == "e1"]
# 終点("e1")で条件をしぼって始点を抽出する
[edges[i][0] for i in range(len(edges)) if edges[i][1] == "e1"]
```

## アルゴリズム

In [4]:
def update_frontier(frontier,edge,edges,counter):
    """
    関数の概要：以前までのフロンティアと獲得エッジからフロンティアを更新する。フロンティアは全ての場合で等しいので、一括計算。
    　　　　　　一般に、１つの頂点で、「入次数の数ー１」だけフロンティアは増える。
    @param frontier ：以前までのフロンティア
    @param edge     ：新規獲得エッジ
    @param edges    ：エッジ集合
    @param counter  ：エッジの入次数を記録している辞書
    @return frontier：更新後のフロンティア
    """
    # 入次数が２で、新しい辺の終点の頂点番号が小さかったら追加するだけ。
    if counter[edge[0]] == 2 and edge[1] == [edges[i][1] for i in range(len(edges)) if edges[i][0] == edge[0]][0]:
        frontier.append(edge[1])
    # 入次数が２で、新しい辺の終点の頂点番号が大きかったら始点を削除して終点を追加する。
    elif counter[edge[0]] == 2 and edge[1] == [edges[i][1] for i in range(len(edges)) if edges[i][0] == edge[0]][1]:
        frontier.remove(edge[0])
        frontier.append(edge[1])
    # 入次数が１の場合は、始点を削除して終点を追加する。
    elif counter[edge[0]] == 1:
        frontier.remove(edge[0])
        frontier.append(edge[1])
    else:
        print("予想外のedge: {}です!!".format(edge))
    
    # 重複を削除する。
    new_frontier = list(set(frontier))
    return new_frontier

In [5]:
# 確認
frontier = ["e0"]
print(frontier)
for i in range(len(edges)):
    frontier = update_frontier(frontier, edges[i], edges, counter)
    print(frontier)

['e0']
['e1', 'e0']
['e1', 'e2']
['e1', 'e3', 'e2']
['e4', 'e3', 'e2']
['e4', 'e3', 'e5']
['e4', 'e5']
['e5']


<img src="../image/graphillion/sample.png" width=60%>

***

In [6]:
def update_mate_arr(mate, frontier, current_edges):
    """
    関数の概要：mate配列を更新する。
    @patam mate         ：以前までのmate配列（dict）
    @param frontier     ：更新後のフロンティア
    @param current_edges：獲得したエッジ集合
    @return new_mate    ：更新後のmate配列（dict）  
    """
    for f in range(len(frontier)):
        # 獲得したエッジ集合から、フロンティアの入次数を求める。
        in_edges  = [current_edges[i][0] for i in range(len(current_edges)) if current_edges[i][1] == frontier[f]]
        N_in  = len(in_edges)
        # 入次数が０だったら、頂点の名前になる。
        if N_in == 0:
            mate[frontier[f]] = frontier[f]
        # 入次数が２(=途中にある)場合は０。
        elif N_in == 2:
            mate[frontier[f]] = "0"
        # 入次数が１つだったら、逆端まで探索する。
        elif N_in == 1:
            edge = in_edges[0]
            b_edge = frontier[f] # １つ前のedgeを記録
            loop = 0             # ループ回数を記録
            # 戻った先で入次数と出次数を調べ続ける。
            while True and loop<len(current_edges): # 全てエッジを通っても終わらない＝ループが起きている。
                in_edges  = [current_edges[i][0] for i in range(len(current_edges)) if current_edges[i][1] == edge]
                out_edges = [current_edges[i][1] for i in range(len(current_edges)) if current_edges[i][0] == edge]
                N_in  = len(in_edges)  # 入次数の数
                N_out = len(out_edges) # 出自数の数
                
                """
                デバッグ用
                
                print("フロンティア:{}".format(frontier[f]))
                print("入次数:{},{}\n出次数:{},{}".format(N_in, in_edges,N_out,out_edges))
                """

                
                # 端っこにたどり着いた場合
                if N_in + N_out == 1:
                    mate[frontier[f]] = edge
                    break
                # 端ではない場合は、端まで探索する。
                elif N_in == 2:
                    tmp = edge
                    edge = [in_edges[i] for i in range(len(in_edges)) if in_edges[i]!=b_edge][0]
                    b_edge = tmp
                elif N_out == 2:
                    tmp = edge
                    edge = [out_edges[i] for i in range(len(out_edges)) if out_edges[i]!=b_edge][0]
                    b_edge = tmp
                elif N_in==1 and N_out==1:
                    tmp_edges = in_edges+out_edges
                    tmp = edge
                    edge = [tmp_edges[i] for i in range(len(tmp_edges)) if tmp_edges[i]!=b_edge][0]
                    b_edge = tmp  
                loop+=1
            
            if loop==len(current_edges):
                for k in range(len(frontier)):
                    mate[frontier[k]] = "0"
                return mate
    return mate

In [7]:
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = 0 
mate_dict

{'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}

In [8]:
# 確認
frontier = ["e2","e3","e4"]
current_edges = [("e0","e2"),("e1","e3"),("e1","e4")]
update_mate_arr(mate_dict, frontier, current_edges)

{'e0': 0, 'e1': 0, 'e2': 'e0', 'e3': 'e4', 'e4': 'e3', 'e5': 0}

<img src="../image/graphillion/sample2.png" width=50%>

***

In [9]:
def One_stroke(count_list):
    """
    関数の概要：一筆書きできるかを判断する。
    @param count_list：それぞれのノードが関わる回数を記録したリスト
    @return flag：できるならTrue, できないならFalse
    """
    flag = False
    for i in range(len(count_list)):
        if i == 0:
            if count_list[i] != 1:
                break
        elif i == len(count_list)-1:
            if count_list[i] != 1:
                break
            flag = True
        else:
            if count_list[i] != 2:
                break
    return flag

In [10]:
# 確認
edges_set = [("e0","e1"),("e1","e3"),("e3","e4")]
counter = []
for i in range(len(edges_set)):
    counter += list(edges_set[i])
count_list = list(collections.Counter(e for e in counter).values())
print("count_list:{}".format(count_list))
print("一筆書き可能" if One_stroke(count_list) else "無理")

count_list:[1, 2, 2, 1]
一筆書き可能


# メインプログラム

__【初期条件】__

In [11]:
# 辺の集合
edges = [("e0","e1"),("e0","e2"),("e1","e3"),("e1","e4"),("e2","e5"),("e3","e5"),("e4","e5")]
# 頂点の集合
vertex_set = ["e0","e1","e2","e3","e4","e5"]
# mate配列
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = "0"
mate_dict["e0"] = "e0"

<img src="../image/graphillion/sample.png" width=60%>

In [12]:
# 入次数を辞書で記録
counter = collections.Counter([edge[0] for edge in edges])

In [13]:
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = "0"
mate_dict["e0"] = "e0" # 初期値をどうするかは悩み中

In [14]:
# フロンティアの初期条件
frontier = ["e0"]

__`[(獲得したedge), 個数, mate配列]`を並べていく。__

In [15]:
edges_set = [[[],1,mate_dict]] # 初期化
for i in range(len(edges)):
    # 新しいノードを獲得する処理
    frontier = update_frontier(frontier, edges[i], edges, counter) # フロンティアを更新する。
    edges_set_get = copy.deepcopy(edges_set) # 新規のエッジを獲得する方
    edges_set_not = copy.deepcopy(edges_set) # 新規のエッジを獲得しない方
    for j in range(len(edges_set_get)): 
        edges_set_get[j][0].append(edges[i])
    edges_set = edges_set_get+edges_set_not 
    
    # mate配列を更新する
    for j in range(len(edges_set)):
        edges_set[j] = [edges_set[j][0],
                        edges_set[j][1],
                        update_mate_arr(edges_set[j][2],frontier,edges_set[j][0])]

    # [mate配列]をkey,[edgeの数,個数,edge]をvalueにする。
    result_dict = dict()
    for j in range(len(edges_set)):
        # フロンティア部分のmate配列。タプルにすることで辞書のキーにできる。
        mate_tuple = (edges_set[j][2][frontier[f]] for f in range(len(frontier))) # 識別子(フロンティアだけ)
        mate_list  =  edges_set[j][2] # mate配列
        N_edges = len(edges_set[j][0]) # 獲得したエッジの数
        get_edges = edges_set[j][0]    # 獲得したエッジの数
        num = edges_set[j][1]          # これより前で圧縮された同じものの数
        if "e0" in mate_tuple:          # e0がないものはゴールできないので、ここで削除する。
            # mate配列の同じものがあれば、エッジ数の少ないものを残して圧縮。
            if mate_tuple in result_dict:
                result_dict[mate_list][1] += num # 同じものはまとめる。
                # エッジ数が少なくなるのであれば、更新する。
                if result_dict[mate_tuple][0] > N_edges:
                    result_dict[mate_tuple][0] = N_edges
                    result_dict[mate_tuple][2] = get_edges
                    result_dict[mate_tuple][3] = mate_list
            else:
                result_dict[mate_tuple] = [N_edges, num, get_edges, mate_list]
    edges_set = []
    for key, value in result_dict.items():
        edges_set.append([value[2],value[1],value[3]])
        
answer = 0
for i in range(len(edges_set)):
    counter = []
    for j in range(len(edges_set[i][0])):
        counter += list(edges_set[i][0][j])
    # 同じ文字が３回以上出ていないかのチェック
    count_list = list(collections.Counter(e for e in counter).values())
    if One_stroke(count_list): # 一筆書きできないものを削除
        answer += edges_set[i][1]

print("目的地への行き方は全部で{}通りです。".format(answer))

目的地への行き方は全部で3通りです。


### 他の例

__【初期条件】__

In [16]:
# 辺の集合
edges = [("e0","e1"),("e0","e3"),("e1","e2"),("e1","e4"),("e3","e4"),("e3","e6"),("e2","e5"),("e4","e5"),("e4","e7"),("e6","e7"),("e5","e8"),("e7","e8")]
# 頂点の集合
vertex_set = ["e0","e1","e2","e3","e4","e5","e6","e7","e8"]
# mate配列
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = "0"
mate_dict["e0"] = "e0"

<img src="../image/graphillion/sample3.png" width=50%>

In [17]:
# 入次数を辞書で記録
counter = collections.Counter([edge[0] for edge in edges])

In [18]:
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = "0"
mate_dict["e0"] = "e0" 

In [19]:
# フロンティアの初期条件
frontier = ["e0"]

In [20]:
edges_set = [[[],1,mate_dict]] # 初期化
for i in range(len(edges)):
    # 新しいノードを獲得する処理
    frontier = update_frontier(frontier, edges[i], edges, counter) # フロンティアを更新する。
    edges_set_get = copy.deepcopy(edges_set) # 新規のエッジを獲得する方
    edges_set_not = copy.deepcopy(edges_set) # 新規のエッジを獲得しない方
    for j in range(len(edges_set_get)): 
        edges_set_get[j][0].append(edges[i])
    edges_set = edges_set_get+edges_set_not
    
    # mate配列を更新する
    for j in range(len(edges_set)):
        edges_set[j] = [edges_set[j][0],
                        edges_set[j][1],
                        update_mate_arr(edges_set[j][2],frontier,edges_set[j][0])]

    # [mate配列]をkey,[edgeの数,個数,edge]をvalueにする。
    result_dict = dict()
    for j in range(len(edges_set)):
        # フロンティア部分のmate配列。タプルにすることで辞書のキーにできる。
        # mate_id = ''.join([edges_set[j][2][frontier[f]] for f in range(len(frontier))]) # 識別子(フロンティアだけを文字化)
        mate_list  =  edges_set[j][2] # mate配列
        mate_id2 = (edges_set[j][2][frontier[f]] for f in range(len(frontier)))
        N_edges = len(edges_set[j][0]) # 獲得したエッジの数
        get_edges = edges_set[j][0]    # 獲得したエッジの数
        num = edges_set[j][1]          # これより前で圧縮された同じものの数
        if "e0" in mate_id2:            # e0がないものはゴールできないので、ここで削除する。
            # mate配列の同じものがあれば、エッジ数の少ないものを残して圧縮。
            if mate_id2 in result_dict:
                result_dict[mate_id2][1] += num # 同じものはまとめる。
                # エッジ数が少なくなるのであれば、更新する。
                if result_dict[mate_id2][0] > N_edges:
                    result_dict[mate_id2][0] = N_edges
                    result_dict[mate_id2][2] = get_edges
                    result_dict[mate_id2][3] = mate_list
            else:
                result_dict[mate_id2] = [N_edges, num, get_edges, mate_list]
    edges_set = []
    for key, value in result_dict.items():
        edges_set.append([value[2],value[1],value[3]])
        
answer = 0
for i in range(len(edges_set)):
    counter = []
    for j in range(len(edges_set[i][0])):
        counter += list(edges_set[i][0][j])
    # 同じ文字が３回以上出ていないかのチェック
    count_list = list(collections.Counter(e for e in counter).values())
    if One_stroke(count_list): # 一筆書きできないものを削除
        answer += edges_set[i][1]

print("目的地への行き方は全部で{}通りです。".format(answer))

目的地への行き方は全部で11通りです。


<img src="../image/graphillion/sample3.png" width=50%>

__寄り道してたどり着くものが消えてしまっている。理由は、一筆書きを調べる方法の問題だった！昇順じゃないとカウントしてないやん！！
まだわかっていない。また、タプルを辞書のキーに利用すると、generaterのidが使われているようで、同じ値を示していても違うものと認識されてしまっていることに注意が必要！__